In [1]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go

In [2]:
from infrastructure.instrument_collection import instrumentCollection

In [3]:
def load_results(ic):
    res = []
    trades = []
    our_curr = ['USD', 'GBP', 'JPY', 'NZD']
    for p1 in our_curr:
        for p2 in our_curr:
            pair = f"{p1}_{p2}"
            if pair in ic.instruments_dict.keys():
                res.append(pd.read_pickle(f"./macd_ema/macd_ema_res_{pair}.pkl"))
                trades.append(pd.read_pickle(f"./macd_ema/trades/macd_ema_trades_{pair}.pkl"))
    return pd.concat(res), pd.concat(trades)

In [4]:
instrumentCollection.LoadInstruments('../data/')

In [ ]:
df_results, df_trades = load_results(instrumentCollection)
df_results.reset_index(drop=True, inplace=True)
df_trades.reset_index(drop=True, inplace=True)


In [ ]:
df_results.head()


In [ ]:
df_trades.head()

In [ ]:
df_results.columns

In [ ]:
# look at the results grouped by setting
gr = df_results.groupby(by=['slow', 'fast', 'ema', 'signal'], as_index=False).sum()

In [ ]:
# sort descending, best result at the top
gr.sort_values(by='result', ascending=False, inplace=True)

In [ ]:
gr.head()

In [ ]:
# for the top setting, let's see what we have in terms of individual currency pairs
df_results_best = df_results[(df_results.ema==50)&(df_results.fast==18)&(df_results.slow==52)&(df_results.signal==9)]

In [ ]:
df_results_best

In [ ]:
# let's see the individual trades for the top setting
df_trades_best = df_trades[(df_trades.ema==50)&(df_trades.fast==18)&(df_trades.slow==52)&(df_trades.signal==9)].copy()

In [ ]:
df_trades_best.columns

In [ ]:
# let's see how long trades last
df_trades_best['duration_h'] = (df_trades_best.end_time - df_trades_best.start_time).dt.total_seconds() / 3600

In [ ]:
df_trades_best['duration_h'].describe()

In [ ]:
# win percentage
wp = (df_trades_best[df_trades_best.result>0].shape[0] / df_trades_best.shape[0]) * 100
print("Win percentage", wp)

In [ ]:
# we will take a look at the cumulative gains, so sort by end_time
df_trades_best.sort_values(by='end_time', inplace=True)

In [ ]:
df_trades_best['CUM_RES'] = df_trades_best.result.cumsum()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_trades_best.end_time,
    y=df_trades_best['CUM_RES'],
    line=dict(width=2)
))

fig.update_yaxes(
    gridcolor="#1f292f"
)
fig.update_xaxes(
    gridcolor="#1f292f"
)

fig.update_layout(
    margin=dict(l=10,r=10,b=10,t=10),
    paper_bgcolor="#2c303c",
    plot_bgcolor="#2c303c",
    font=dict(size=8, color="#e1e1e1"),
    height=400
)

fig.show()